<div class="alert alert-block alert-info">
<b>How to run this notebook?</b><br />
<ol>
    <li>Install the DockStream environment: conda env create -f environment.yml in the DockStream directory</li>
    <li>Activate the environment: conda activate DockStreamCommunity</li>
    <li>Execute jupyter: jupyter notebook</li>
    <li> Copy the link to a browser</li>
    <li> Update variables <b>dockstream_path</b> and <b>dockstream_env</b> (the path to the environment DockStream) in the 
        first code block below</li>
    </ol>
</div>

# Command-Line Interfaces
While `DockStream` can be loaded as a library, the most common use-case is to call a command-line interface (e.g. implicitly when using `REINVENT`). The entry points described below are currently supported and are explained together with the available parameters. To execute them, we need to either load the `conda` environment before or specify the full path to the `python` version we want to use (here, we will make use of the latter option).

In [1]:
import os
import json
import tempfile

# update these paths to reflect your system's configuration
dockstream_path = os.path.expanduser("~/Desktop/ProjectData/DockStream")
dockstream_env = os.path.expanduser("~/miniconda3/envs/DockStream")
dockstreamcommunity_env = os.path.expanduser("~/miniconda3/envs/DockStreamCommunity")

# generate the paths to the entry points
target_preparator = os.path.join(dockstream_path, "target_preparator.py")
docker = os.path.join(dockstream_path, "docker.py")
sdf2smiles = os.path.join(dockstream_path, "sdf2smiles.py")
unit_tests = os.path.join(dockstream_path, "unit_tests.py")
benchmarking = os.path.join(dockstream_path, "benchmarking.py")
analysis = os.path.join(dockstream_path, "analysis.py")

## General Options
Most command-line interfaces support the `-h` (or `--help`) flag, which will cause the script to ignore all other options set and print a usage description. Another common option is `-log_conf`, which is optional and allows to set the logging configuration (see `config/logging` subfolder of `DockStream` for examples). By default, only important messages (`Error` and `Warning` levels) are logged out, but by using the `-log_conf` parameter, users can utilize their own definitions. The recommended way, however, to obtain more detailed logging messages (for example when debugging a run), is to use the `-debug` flag.

## `target_preparator.py`
For all backends we need to define the target in a specific format. Depending on the backend, these files hold information on the location of the binding site, constraintes, etc. - except for `Glide`, we can use this command line tool to generate the receptors (please see the respective notebooks for details). While the major input is a `JSON` file defining the preparation run (set by the only mandatory parameter, `-conf`), you can tweak the behaviour also by setting additional parameters.

In [2]:
# execute this in a command-line environment after replacing the parameters
!{dockstream_env}/bin/python {target_preparator} -h

usage: target_preparator.py [-h] [-conf CONF] [-validation VALIDATION]
                            [-silent SILENT] [-debug] [-log_conf LOG_CONF]

Implements entry point for the target preparation for one or multiple
backends.

optional arguments:
  -h, --help            show this help message and exit
  -conf CONF            A path to an preparation configuration file (JSON
                        dictionary) that is to be executed.
  -validation VALIDATION
                        If set to False, this flag will prohibit a JSON Schema
                        validation.
  -silent SILENT        If set, the program will silently execute without
                        printing status updates.
  -debug                Set this flag to activate the inbuilt debug logging
                        mode (this will overwrite parameter "-log_conf", if
                        set).
  -log_conf LOG_CONF    Set absolute path to a logger configuration other than
                        the default st

## `unit_tests.py`
This entry point does not have any parameters but it might be a good idea to check any new installation by executing:

```
<path_to_environment>/python unit_tests.py
```

**Note:** Some of them might fail if they require a software dependency that is not found or not installed on your system.

## `docker.py`
This is the main component of `DockStream` and performs its tasks based on the instruction `JSON` file provided with the `-conf` parameter. Note, that the `-validation` parameter is not yet functional.

In [3]:
# execute this in a command-line environment after replacing the parameters
!{dockstream_env}/bin/python {docker} -h

usage: docker.py [-h] -conf CONF [-validation VALIDATION] [-silent SILENT]
                 [-smiles SMILES] [-print_scores] [-print_all] [-debug]
                 [-log_conf LOG_CONF] [-output_prefix OUTPUT_PREFIX]
                 [-input_csv INPUT_CSV]
                 [-input_csv_smiles_column INPUT_CSV_SMILES_COLUMN]
                 [-input_csv_names_column INPUT_CSV_NAMES_COLUMN]

Implements entry point for the docking using one or multiple backends.

optional arguments:
  -h, --help            show this help message and exit
  -conf CONF            A path to an docking configuration file (JSON
                        dictionary) that is to be executed.
  -validation VALIDATION
                        If set to False, this flag will prohibit a JSON Schema
                        validation.
  -silent SILENT        If set, the program will silently execute without
                        printing status updates.
  -smiles SMILES        Use this flag to hand over the input SMILES 

The interface to `REINVENT` is facilitated via the `-smiles` and `-print_scores` options (for input and output, respectively). In addition, `-output_prefix` is used to add the epoch number from `REINVENT` to the file names. Option `-print_scores` (without `-print_all` activated) causes `DockStream` to print the best score per ligand (not conformation nor enumeration) to `stdout`.

## `sdf2smiles.py`
This is a simple convenience script that takes an `SDF` file as input and produces a text file with one `SMILE` per line for each molecule. This can be directly used as input for `DockStream`.

In [4]:
!{dockstream_env}/bin/python {sdf2smiles} -h

usage: sdf2smiles.py [-h] [-sdf SDF] [-smi SMI] [-csv CSV] [-keep_stereo]
                     [-tags2columns TAGS2COLUMNS [TAGS2COLUMNS ...]]

Implements simple translator taking an SDF file and spitting out SMILES.

optional arguments:
  -h, --help            show this help message and exit
  -sdf SDF              A path a SDF file.
  -smi SMI              A path an output text file.
  -csv CSV              A path an output CSV file.
  -keep_stereo          If set, exported SMILES contain stereo-information.
  -tags2columns TAGS2COLUMNS [TAGS2COLUMNS ...]
                        A list of strings for which tags should be transformed
                        into columns.


## `benchmarking.py`
This script facilitates batch execution of `DockStream`. A folder containing any number of configuration `JSONs` can be passed to the `-input_path` parameter. See `demo_Benchmarking_Script` for details.

In [5]:
# execute this in a command-line environment after replacing the parameters
!{dockstream_env}/bin/python {benchmarking} -h

usage: benchmarking.py [-h] -input_path INPUT_PATH

Facilitates batch DockStream execution.

optional arguments:
  -h, --help            show this help message and exit
  -input_path INPUT_PATH
                        The path to either a folder of DockStream json files
                        or a single json file.


## `analysis.py`
This script automates analysis of `DockStream` results. A configuration `JSON` can be passed to the `-input_json` parameter to denote the analysis mode. See `demo_Analysis_Script` for details.

In [6]:
# execute this in a command-line environment after replacing the parameters
!{dockstreamcommunity_env}/bin/python {analysis} -h

usage: analysis.py [-h] -input_json INPUT_JSON

Implements entry point to DockStream output analysis.

optional arguments:
  -h, --help            show this help message and exit
  -input_json INPUT_JSON
                        Path to user provided json file which contains all the
                        paths/metrics for analysis.
